# Dataset Routes


In [1]:
# | default_exp routes.dataset

In [2]:
# | exporti
from typing import Optional, List
from enum import Enum

import io
import pandas as pd

import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [3]:
# | hide
import os

## Query Datasets


In [4]:
# | export
class DatasetNotFoundError(de.DomoError):
    def __init__(
        self,
        dataset_id,
        domo_instance,
        status=None,
        parent_class=None,
        function_name=None,
    ):
        message = f"dataset - {dataset_id} not found"

        super().__init__(
            message,
            status=status,
            domo_instance=domo_instance,
            function_name=function_name,
            parent_class=parent_class,
        )

In [5]:
# | export


class QueryRequestError(de.DomoError):
    def __init__(
        self,
        dataset_id,
        domo_instance,
        sql,
        status=None,
        message="",
        parent_class=None,
        function_name=None,
    ):
        message = f"dataset - {dataset_id} received a bad request {message}.  Check your SQL \n {sql}"

        super().__init__(
            message,
            status=status,
            domo_instance=domo_instance,
            parent_class=parent_class,
            function_name=function_name,
        )


# typically do not use
async def query_dataset_public(
    dev_auth: dmda.DomoDeveloperAuth,
    dataset_id: str,
    sql: str,
    session: httpx.AsyncClient,
    debug_api: bool = False,
):
    """query for hitting public apis, requires client_id and secret authentication"""

    url = f"https://api.domo.com/v1/datasets/query/execute/{dataset_id}?IncludeHeaders=true"

    body = {"sql": sql}

    return await gd.get_data(
        auth=dev_auth,
        url=url,
        method="POST",
        body=body,
        session=session,
        debug_api=debug_api,
    )


async def query_dataset_private(
    auth: dmda.DomoAuth,  # DomoFullAuth or DomoTokenAuth
    dataset_id: str,
    sql: str,
    session: Optional[httpx.AsyncClient] = None,
    loop_until_end: bool = False,  # retrieve all available rows
    limit=100,  # maximum rows to return per request.  refers to PAGINATION
    skip=0,
    maximum=100,  # equivalent to the LIMIT or TOP clause in SQL, the number of rows to return total
    filter_pdp_policy_id_ls: List[int] = None,
    debug_api: bool = False,
    debug_loop: bool = False,
    timeout: int = 10,
    parent_class=None,
    debug_num_stacks_to_drop=1,
):
    """execute SQL queries against private APIs, requires DomoFullAuth or DomoTokenAuth"""

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/execute/{dataset_id}"

    offset_params = {
        "offset": "offset",
        "limit": "limit",
    }

    # def body_fn(skip, limit):
    #     return {"sql": f"{sql} limit {limit} offset {skip}"}

    def body_fn(skip, limit, body=None):
        body = {"sql": f"{sql} limit {limit} offset {skip}"}

        if filter_pdp_policy_id_ls:
            body.update(
                {
                    "context": {
                        "dataControlContext": {
                            "filterGroupIds": filter_pdp_policy_id_ls,
                            "previewPdp": True,
                        }
                    }
                }
            )

        return body

    def arr_fn(res) -> pd.DataFrame:
        rows_ls = res.response.get("rows")
        columns_ls = res.response.get("columns")
        output = []
        for row in rows_ls:
            new_row = {}
            for index, column in enumerate(columns_ls):
                new_row[column] = row[index]
            output.append(new_row)
            # pd.DataFrame(data=res.response.get('rows'), columns=res.response.get('columns'))
        return output

    res = await gd.looper(
        auth=auth,
        method="POST",
        url=url,
        arr_fn=arr_fn,
        offset_params=offset_params,
        limit=limit,
        skip=skip,
        maximum=maximum,
        session=session,
        body_fn=body_fn,
        debug_api=debug_api,
        debug_loop=debug_loop,
        loop_until_end=loop_until_end,
        timeout=timeout,
        parent_class=parent_class,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if res.status == 404 and res.response == "Not Found":
        raise DatasetNotFoundError(
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    if res.status == 400 and res.response == "Bad Request":
        raise QueryRequestError(
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            sql=sql,
            status=res.status,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    if not res.is_success:
        raise QueryRequestError(
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            sql=sql,
            message=res.response,
            status=res.status,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [6]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


dataset_id = os.environ["DOJO_DATASET_ID"]


sql = f"SELECT * FROM TABLE"

ds_res = await query_dataset_private(
    dataset_id=dataset_id,
    auth=token_auth,
    sql=sql,
    skip=0,
    limit=1000,
    filter_pdp_policy_id_ls=[1225, 1226],  # to apply pdp filter context
    loop_until_end=True,
    debug_api=False,
)
print(len(ds_res.response))
pd.DataFrame(ds_res.response)

warning this token has not been validated by who_am_i, run get_auth_token first
2


,objectID,url,Title,article,views,created_dt,published_dt
0,000005034,https://domo-support.domo.com/s/article/360042...,Adding Scale Markers to Your Charts,IntroDomo provides the ability to set scale ma...,50,2022-11-02T21:00:00,2022-11-02T21:04:00
1,000004347,https://domo-support.domo.com/s/article/457779...,Accessing Goals Data,"IntroIn Goals, you can see the overall status ...",23,2022-10-24T21:41:00,2022-10-24T22:39:00


## Dataset Properties


In [7]:
# | export
async def get_dataset_by_id(
    dataset_id: str,  # dataset id from URL
    auth: Optional[dmda.DomoAuth] = None,  # requires full authentication
    debug_api: bool = False,  # for troubleshooting API request
    session: Optional[httpx.AsyncClient] = None,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
) -> rgd.ResponseGetData:  # returns metadata about a dataset
    """retrieve dataset metadata"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if res.status == 404 and res.response == "Not Found":
        raise DatasetNotFoundError(
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [8]:
import os
import pandas as pd

try:
    token_auth = dmda.DomoTokenAuth(
        domo_instance="domo-community",
        domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    )

    await get_dataset_by_id(dataset_id=123, auth=token_auth)

except DatasetNotFoundError as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  DatasetNotFoundError 🛑 - function: get_dataset_by_id || dataset - 123 not found || status 404 || error at domo-community


In [9]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

ds_res = await get_dataset_by_id(
    dataset_id=os.environ["DOJO_DATASET_ID"], auth=token_auth
)
pd.DataFrame([ds_res.response])

warning this token has not been validated by who_am_i, run get_auth_token first


ConnectTimeout: 

In [ ]:
# | export
async def get_schema(
    auth: dmda.DomoAuth,
    dataset_id: str,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class=None,
) -> rgd.ResponseGetData:
    """retrieve the schema for a dataset"""

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/datasources/{dataset_id}/schema/indexed?includeHidden=false"

    return await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### sample implementation of get_schema


In [ ]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_schema(dataset_id=os.environ["DOJO_DATASET_ID"], auth=token_auth)
# retrieve schema from response
pd.DataFrame(res.response.get("tables")[0].get("columns"))

In [ ]:
# | export
async def alter_schema(
    auth: dmda.DomoAuth,
    schema_obj: dict,
    dataset_id: str,
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
) -> rgd.ResponseGetData:
    """retrieve the schema for a dataset"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v2/datasources/{dataset_id}/schemas"

    return await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body=schema_obj,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

In [ ]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

schema_res = await get_schema(dataset_id=os.environ["DOJO_DATASET_ID"], auth=token_auth)

schema_obj = schema_res.response["tables"][0]

await alter_schema(
    dataset_id=os.environ["DOJO_DATASET_ID"], auth=token_auth, schema_obj=schema_obj
)

## must index dataset after alter schema
# await index_dataset(dataset_id=os.environ["DOJO_DATASET_ID"], auth=token_auth )

In [ ]:
# | export
async def set_dataset_tags(
    auth: dmda.DomoFullAuth,
    tag_ls: List[str],  # complete list of tags for dataset
    dataset_id: str,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
    return_raw: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
):
    """REPLACE tags on this dataset with a new list"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/ui/v3/datasources/{dataset_id}/tags"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        debug_api=debug_api,
        body=tag_ls,
        session=session,
        return_raw=return_raw,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    if res.status == 200:
        res.set_response(
            response=f'Dataset {dataset_id} tags updated to [{ ", ".join(tag_ls) }]'
        )

    return res

In [ ]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

tag_ls = ["hackercore", "developer_documentation"]

await set_dataset_tags(
    auth=token_auth,
    tag_ls=tag_ls,
    dataset_id=os.environ["DOJO_DATASET_ID"],
    debug_api=False,
    return_raw=False,
)

## Upload Data

#### overview

In the URL, parts refers to the multi-part API and is unrelated to the partitions concept. The multi-part API was designed to allow sending multiple streams of Data into a data_version simultaneously.

In stage 1, the values passed in the Body will be superseded by values in the COMMIT (stage 3), so best practices is to not populate values here.

The response includes an uploadId, which must be stored and passed to the URL of the subsequent upload request (stages 2 and 3).

#### url params

The dataTag parameter allows users to UPDATE or REPLACE a datatag (partition)

NOTE: restateDataTag is largely deprecated // exists for backward compatibility

#### body params

The appendId parameter accepts "latest" or "None"

latest will APPEND the data version to the dataset


In [ ]:
# | export
class UploadDataError(de.DomoError):
    """raise if unable to upload data to Domo"""

    def __init__(
        self, stage_num: int, dataset_id: str, status, message, domo_instance: str
    ):
        message = f"error uploading data during Stage { stage_num} - {message}"

        super().__init__(
            entity_id=dataset_id,
            message=message,
            status=status,
            domo_instance=domo_instance,
        )

In [ ]:
# | export
async def upload_dataset_stage_1(
    auth: dmda.DomoAuth,
    dataset_id: str,
    #  restate_data_tag: str = None, # deprecated
    partition_tag: str = None,  # synonymous with data_tag
    session: Optional[httpx.AsyncClient] = None,
    debug_api: bool = False,
    return_raw: bool = False,
) -> rgd.ResponseGetData:
    """preps dataset for upload by creating an upload_id (upload session key) pass to stage 2 as a parameter"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}/uploads"

    # base body assumes no paritioning
    body = {"action": None, "appendId": None}

    params = None

    if partition_tag:
        # params = {'dataTag': restate_data_tag or data_tag} # deprecated
        params = {"dataTag": partition_tag}
        body.update({"appendId": "latest"})

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body=body,
        session=session,
        debug_api=debug_api,
        params=params,
    )

    if not res.is_success:
        raise UploadDataError(
            stage_num=1,
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
        )

    if return_raw:
        return res

    upload_id = res.response.get("uploadId")

    if not upload_id:
        raise UploadDataError(
            stage_num=1,
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            message="no upload_id",
        )

    res.response = upload_id

    return res

In [ ]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

dataset_id = "b102e530-6472-4cc3-b420-d5ab3792fdf9"
partition_key = "2023-04-27"

await upload_dataset_stage_1(
    auth=token_auth, dataset_id=dataset_id, partition_tag=partition_key, debug_api=False
)

In [ ]:
# | export
async def upload_dataset_stage_2_file(
    auth: dmda.DomoAuth,
    dataset_id: str,
    upload_id: str,  # must originate from  a stage_1 upload response
    data_file: Optional[io.TextIOWrapper] = None,
    session: Optional[httpx.AsyncClient] = None,
    # only necessary if streaming multiple files into the same partition (multi-part upload)
    part_id: str = 2,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}/uploads/{upload_id}/parts/{part_id}"

    body = data_file

    res = await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        content_type="text/csv",
        body=body,
        session=session,
        debug_api=debug_api,
    )

    if not res.is_success:
        raise UploadDataError(
            stage_num=2,
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
        )

    res.upload_id = upload_id
    res.dataset_id = dataset_id
    res.part_id = part_id

    return res

In [ ]:
# | export


async def upload_dataset_stage_2_df(
    auth: dmda.DomoAuth,
    dataset_id: str,
    upload_id: str,  # must originate from  a stage_1 upload response
    upload_df: pd.DataFrame,
    session: Optional[httpx.AsyncClient] = None,
    part_id: str = 2,  # only necessary if streaming multiple files into the same partition (multi-part upload)
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}/uploads/{upload_id}/parts/{part_id}"

    body = upload_df.to_csv(header=False, index=False)

    # if debug:
    #     print(body)

    res = await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        content_type="text/csv",
        body=body,
        session=session,
        debug_api=debug_api,
    )

    if not res.is_success:
        raise UploadDataError(
            stage_num=2,
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
        )

    res.upload_id = upload_id
    res.dataset_id = dataset_id
    res.part_id = part_id

    return res

In [ ]:
# | export
async def upload_dataset_stage_3(
    auth: dmda.DomoAuth,
    dataset_id: str,
    upload_id: str,  # must originate from  a stage_1 upload response
    session: Optional[httpx.AsyncClient] = None,
    update_method: str = "REPLACE",  # accepts REPLACE or APPEND
    #  restate_data_tag: str = None, # deprecated
    partition_tag: str = None,  # synonymous with data_tag
    is_index: bool = False,  # index after uploading
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    """commit will close the upload session, upload_id.  this request defines how the data will be loaded into Adrenaline, update_method
    has optional flag for indexing dataset.
    """

    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}/uploads/{upload_id}/commit"

    body = {"index": is_index, "action": update_method}

    if partition_tag:
        body.update(
            {
                "action": "APPEND",
                #  'dataTag': restate_data_tag or data_tag,
                #  'appendId': 'latest' if (restate_data_tag or data_tag) else None,
                "dataTag": partition_tag,
                "appendId": "latest" if partition_tag else None,
                "index": is_index,
            }
        )

    res = await gd.get_data(
        auth=auth,
        method="PUT",
        url=url,
        body=body,
        session=session,
        debug_api=debug_api,
    )

    if not res.is_success:
        raise UploadDataError(
            stage_num=3,
            dataset_id=dataset_id,
            domo_instance=auth.domo_instance,
            status=res.status,
            message=res.response,
        )

    res.upload_id = upload_id
    res.dataset_id = dataset_id

    return res

In [ ]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

ds_id = "cbae0e0c-a92d-4a4c-8d0c-c9ccd38fe928"

# await get_schema(dataset_id= ds_id, auth=token_auth)

df = pd.DataFrame([{"col_a": "a", "col_b": "b", "col_c": "c"}])


s1_res = await upload_dataset_stage_1(
    auth=token_auth, dataset_id=ds_id, partition_tag=None, debug_api=False
)

upload_id = s1_res.response
upload_id

s2_res = await upload_dataset_stage_2_df(
    auth=token_auth,
    dataset_id=ds_id,
    upload_id=upload_id,
    upload_df=df,
    part_id=2,
    debug_api=False,
)


s3_res = await upload_dataset_stage_3(
    auth=token_auth,
    dataset_id=ds_id,
    upload_id=upload_id,
    update_method="REPLACE",  # accepts REPLACE or APPEND
    is_index=True,  # index after uploading
)

s3_res.is_success

In [ ]:
# | export


async def index_dataset(
    auth: dmda.DomoAuth,
    dataset_id: str,
    session: Optional[httpx.AsyncClient] = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    """manually index a dataset"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}/indexes"

    body = {"dataIds": []}

    return await gd.get_data(
        auth=auth,
        method="POST",
        body=body,
        url=url,
        session=session,
        debug_api=debug_api,
    )

In [ ]:
# | export
async def index_status(
    auth: dmda.DomoAuth,
    dataset_id: str,
    index_id: str,
    session: Optional[httpx.AsyncClient] = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    """get the completion status of an index"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}/indexes/{index_id}/statuses"

    return await gd.get_data(
        auth=auth, method="GET", url=url, session=session, debug_api=debug_api
    )

## Working with Partitions


In [ ]:
# | export
def generate_list_partitions_body(limit=100, offset=0):
    return {
        "paginationFields": [
            {
                "fieldName": "datecompleted",
                "sortOrder": "DESC",
                "filterValues": {"MIN": None, "MAX": None},
            }
        ],
        "limit": limit,
        "offset": offset,
    }


async def list_partitions(
    auth: dmda.DomoAuth,
    dataset_id: str,
    body: dict = None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_loop: bool = False,
):
    body = body or generate_list_partitions_body()

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/datasources/{dataset_id}/partition/list"

    offset_params = {
        "offset": "offset",
        "limit": "limit",
    }

    def arr_fn(res) -> list[dict]:
        return res.response

    res = await gd.looper(
        auth=auth,
        method="POST",
        url=url,
        arr_fn=arr_fn,
        body=body,
        offset_params_in_body=True,
        offset_params=offset_params,
        loop_until_end=True,
        session=session,
        debug_loop=debug_loop,
        debug_api=debug_api,
    )

    if res.status == 404 and res.response == "Not Found":
        raise DatasetNotFoundError(
            dataset_id=dataset_id, domo_instance=auth.domo_instance, status=res.status
        )
    return res

In [ ]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

dataset_id = "d2b21660-4ba8-400c-badf-aeef5a9abae1"

res = await list_partitions(auth=token_auth, dataset_id=dataset_id)

ds_partition_ls = res.response

pd.DataFrame(ds_partition_ls[0:5])

In [ ]:
# | export
def generate_create_dataset_body(
    dataset_name: str, dataset_type: str = "API", schema: dict = None
):
    schema = schema or {
        "columns": [
            {"type": "STRING", "name": "Friend"},
            {"type": "STRING", "name": "Attending"},
        ]
    }

    return {
        "userDefinedType": dataset_type,
        "dataSourceName": dataset_name,
        "schema": schema,
    }


async def create(
    auth: dmda.DomoAuth,
    dataset_name: str,
    dataset_type: str = "api",
    session: httpx.AsyncClient = None,
    schema: dict = None,
    debug_api: bool = False,
):
    body = generate_create_dataset_body(
        dataset_name=dataset_name, dataset_type=dataset_type, schema=schema
    )

    url = f"https://{auth.domo_instance}.domo.com/api/data/v2/datasources"

    return await gd.get_data(
        auth=auth,
        method="POST",
        url=url,
        body=body,
        session=session,
        debug_api=debug_api,
    )

#### sample implementation of create dataset


In [ ]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# await create(dataset_name = 'hello world', dataset_type = 'api', auth = token_auth)

In [ ]:
# | export
async def delete_partition_stage_1(
    auth: dmda.DomoAuth,
    dataset_id: str,
    dataset_partition_id: str,
    debug_api: bool = False,
):
    # Delete partition has 3 stages
    # Stage 1. This marks the data version associated with the partition tag as deleted.  It does not delete the partition tag or remove the association between the partition tag and data version.  There should be no need to upload an empty file – step #3 will remove the data from Adrenaline.
    # update on 9/9/2022 based on the conversation with Greg Swensen
    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/datasources/{dataset_id}/tag/{dataset_partition_id}/data"

    return await gd.get_data(auth=auth, method="DELETE", url=url, debug_api=debug_api)


# Stage 2. This will remove the partition association so that it doesn’t show up in the list call.  Technically, this is not required as a partition against a deleted data version will not count against the 400 partition limit, but as the current partitions api doesn’t make that clear, cleaning these up will make it much easier for you to manage.

In [ ]:
# | export
async def delete_partition_stage_2(
    auth: dmda.DomoAuth,
    dataset_id: str,
    dataset_partition_id: str,
    debug_api: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/datasources/{dataset_id}/partition/{dataset_partition_id}"

    return await gd.get_data(auth=auth, method="DELETE", url=url, debug_api=debug_api)

In [ ]:
# | export
async def delete(
    auth: dmda.DomoAuth,
    dataset_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}?deleteMethod=hard"

    return await gd.get_data(
        auth=auth, method="DELETE", url=url, session=session, debug_api=debug_api
    )

In [ ]:
# | export


class ShareDataset_AccessLevelEnum(Enum):
    CO_OWNER = "CO_OWNER"
    CAN_EDIT = "CAN_EDIT"
    CAN_SHARE = "CAN_SHARE"


def generate_share_dataset_payload(
    entity_type,  # USER or GROUP
    entity_id,
    access_level: ShareDataset_AccessLevelEnum = ShareDataset_AccessLevelEnum.CAN_SHARE,
    is_send_email: bool = False,
):
    return {
        "permissions": [
            {"type": entity_type, "id": entity_id, "accessLevel": access_level.value}
        ],
        "sendEmail": is_send_email,
    }

In [ ]:
# | export
class ShareDataset_Error(de.DomoError):
    def __init__(
        self,
        dataset_id,
        status,
        response,
        domo_instance,
        parent_class=None,
        function_name=None,
    ):
        message = f"error sharing dataset {dataset_id} - {response}"

        super().__init__(
            status=status,
            domo_instance=domo_instance,
            message=message,
            parent_class=parent_class,
            function_name=function_name,
        )


async def share_dataset(
    auth: dmda.DomoAuth,
    dataset_id: str,
    body: dict,
    session: httpx.AsyncClient = None,
    debug_api=False,
    parent_class=None,
    debug_num_stacks_to_drop=1,
):
    url = f"https://{auth.domo_instance}.domo.com/api/data/v3/datasources/{dataset_id}/share"

    res = await gd.get_data(
        auth=auth,
        method="POST",
        url=url,
        body=body,
        session=session,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise ShareDataset_Error(
            dataset_id=dataset_id,
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
        )

    update_user_ls = [f"{user['type']} - {user['id']}" for user in body["permissions"]]

    res.response = (
        f"updated access list { ', '.join(update_user_ls)} added to {dataset_id}"
    )
    return res

In [ ]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


body = generate_share_dataset_payload(
    entity_type="USER",
    entity_id=os.environ["DOMO_DOJO_USER_ID"],
    #    access_level=ShareDataset_EntityTypes_Enum.CO_OWNER
)

dataset_id = "d2b21660-4ba8-400c-badf-aeef5a9abae1"

try:
    print(await share_dataset(dataset_id=dataset_id, body=body, auth=token_auth))
except Exception as e:
    print(e)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()